<a href="https://colab.research.google.com/github/JGibs63/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
pip install tensorflow

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
print(application_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB
None


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['NAME', 'EIN'], inplace=True)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [5]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
unique_counts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
value_counts = application_df['APPLICATION_TYPE'].value_counts()
value_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 100
application_types_to_replace = value_counts[value_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
class_value_counts = application_df['CLASSIFICATION'].value_counts()
class_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_value_counts_over_1 = class_value_counts[class_value_counts > 1]
class_value_counts_over_1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = class_value_counts[class_value_counts < cutoff_value].index.tolist()
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df)
dummies_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [12]:
# Split our preprocessed data into our features and target arrays
y = dummies_df['IS_SUCCESSFUL'].values
X = dummies_df.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Optimize the Model from Notebook

In [26]:
# Add more nodes from initial attempt
input_features_total = len(X_train[0])
hidden_nodes_layer1 = 6
hidden_nodes_layer2 = 4

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features_total, activation = "relu"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 6)                 306       
                                                                 
 dense_10 (Dense)            (None, 4)                 28        
                                                                 
 dense_11 (Dense)            (None, 1)                 5         
                                                                 
Total params: 339 (1.32 KB)
Trainable params: 339 (1.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model (Optimization Attempt #1)
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 2s 1ms/step - loss: 0.6308 - accuracy: 0.6787
Epoch 2/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5748 - accuracy: 0.7177
Epoch 3/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5675 - accuracy: 0.7194
Epoch 4/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5637 - accuracy: 0.7203
Epoch 5/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5606 - accuracy: 0.7207
Epoch 6/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5575 - accuracy: 0.7223
Epoch 7/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5527 - accuracy: 0.7275
Epoch 8/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5509 - accuracy: 0.7290
Epoch 9/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5496 - accuracy: 0.7285
Epoch 10/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5490 - accura

In [23]:
# Evaluate the model using the test data
print(f'Optimization Attempt #1 results:')
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Optimization Attempt #1 results:
268/268 - 0s - loss: 0.5510 - accuracy: 0.7335 - 397ms/epoch - 1ms/step
Loss: 0.5510200262069702, Accuracy: 0.7335277199745178


In [49]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.4 MB/s eta 0:00:00


In [67]:
import kerastuner as kt

def create_model(hp):
    nn_model_two = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model_two.add(tf.keras.layers.Dense(units=hp.Int('first_units',
    min_value=1,
    max_value=20,
    step=2), activation=activation, input_dim=50))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model_two.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=20,
            step=2),
            activation=activation))

    nn_model_two.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    nn_model_two.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model_two

# Create the model so that nn_model_two can be accessed outside of the function
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=150,
    hyperband_iterations=2)

Reloading Tuner from ./untitled_project/tuner0.json


In [71]:
# Run the kerastuner search for best hyperparameters (Optimization Attempt #2)
tuner.search(X_train_scaled,y_train,epochs=156,validation_data=(X_test_scaled,y_test))

Trial 155 Complete [00h 00m 12s]
val_accuracy: 0.7292128205299377

Best val_accuracy So Far: 0.7330612540245056
Total elapsed time: 00h 29m 52s

Search: Running Trial #156

Value             |Best Value So Far |Hyperparameter
relu              |tanh              |activation
7                 |3                 |first_units
2                 |1                 |num_layers
17                |5                 |units_0
19                |11                |units_1
9                 |5                 |units_2
1                 |19                |units_3
13                |11                |units_4
11                |3                 |units_5
6                 |17                |tuner/epochs
0                 |6                 |tuner/initial_epoch
3                 |4                 |tuner/bracket
0                 |2                 |tuner/round

Epoch 1/6
804/804 [==============================] - 3s 3ms/step - loss: 0.5922 - accuracy: 0.7040 - val_loss: 0.5673 - val_accuracy: 0.72

KeyboardInterrupt: 

In [72]:
# Evaluate the model using the test data
print(f'Optimization Attempt #2 results:')
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Optimization Attempt #2 results:
268/268 - 0s - loss: 0.5538 - accuracy: 0.7331 - 335ms/epoch - 1ms/step
Loss: 0.5538209080696106, Accuracy: 0.7330612540245056


In [76]:
# Use Deep Learning Model for Attempt #3
nn_model_three = tf.keras.models.Sequential()
nn_model_three.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=50))  # Adjust input_dim to 50
nn_model_three.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model_three.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [77]:
# Compile the Deep Learning model together and customize metrics
nn_model_three.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [78]:
# Train the model
fit_model = nn_model_three.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 2s 1ms/step - loss: 0.5952 - accuracy: 0.6988
Epoch 2/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5578 - accuracy: 0.7254
Epoch 3/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5536 - accuracy: 0.7274
Epoch 4/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5515 - accuracy: 0.7289
Epoch 5/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5502 - accuracy: 0.7299
Epoch 6/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5490 - accuracy: 0.7306
Epoch 7/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5479 - accuracy: 0.7313
Epoch 8/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5476 - accuracy: 0.7311
Epoch 9/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5469 - accuracy: 0.7320
Epoch 10/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5460 - accura

In [79]:
# Evaluate the model using the test data
print(f'Optimization Attempt #3 results:')
model_loss, model_accuracy = nn_model_three.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Optimization Attempt #3 results:
268/268 - 0s - loss: 0.5580 - accuracy: 0.7259 - 294ms/epoch - 1ms/step
Loss: 0.5579513907432556, Accuracy: 0.7259474992752075


In [93]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity_Optimization.h5')

In [94]:
from google.colab import files
files.download('AlphabetSoupCharity_Optimization.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>